## Mancala rules

* Players sit on opposite sides of the long edge of the board
* There are 6 small pits in the middle of the board and 2 large ones at each end.  The small ones in the middle and the large pit on your right are yours.  The small ones on the other side and the large pit to your opponent's right are theirs
* The large pits at the end of the board are called Mancalas
* Set up the board with 4 stones per small pit (none in the mancalas)
* On every turn, select a pit on your side of the board that contains one or more stones,  then distribute its stones, one stone per pit, in an counter-clockwise direction until you have no stones remaining
* If you encounter your opponent's mandala, skip it
* If you encounter your mancala, drop a stone into it
* If the last stone lands in an empty pit on your side of the board, capture this stone and any stones in your opponent's pit on the other side of the board, collect all of these stones, including the one that just landed, and place them into your mancala.
* If either player's pits are entirely empty, the game concludes. 
* The player who still has stones on his side of the board when the game concludes places all of these pieces into their mancala.
The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.


In [21]:
import random
from games import *
from utils import vector_add

In [86]:
# From AIMA repository. Added a parameter 'max_depth'
def alpha_beta_search(state, game, max_depth = 5):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth-1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth-1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, max_depth-1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


In [87]:
class Mancala(Game):
    def __init__(self, pits_per_player = 6, stones_per_pit = 4):
        """
        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
        
        # Needed for report
        self.p1_turn_amount = 0
        self.p2_turn_amount = 0
        
        # For game class -> GameState declaration given in games.py
        self.initial = GameState(to_move=self.current_player, utility=0, board=self.board, moves = self.moves)
        
    
    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        actions = []
        if state.to_move == 1:
            lower, upper = self.p1_pits_index
        else:
            lower, upper = self.p2_pits_index
                
        for i in range(lower, upper + 1):
            if state.board[i] > 0: # Check if pit is non-empty
                actions.append(i - lower + 1) # Convert back to 1–6 numbering

        return actions
            
    def result(self, state, move):
        """
        Return the GameState that results from applying 'move' to 'state'.
        This is essentially a 'what-if' simulator for the AI. Thus we need to make a copy
        of the game so we don't change our real game board
        """
        # Make a copy of the entire game 
        temp = copy.deepcopy(self)
        
        # Add values from parameters to copy 
        temp.board = state.board.copy()
        temp.current_player = state.to_move
        temp.moves = state.moves.copy()
        
        # Execute the given move on the copy
        temp.play(move)
        
        # Return a new GameState from the modified copy
        return GameState(
            to_move = temp.current_player,
            utility = self.utility( GameState(state.to_move, 0, temp.board, temp.moves),state.to_move),
            board = temp.board.copy(),
            moves = temp.moves.copy())
    
    def utility(self, state, player):
        """
        Return the value of this final state to player. We are using 
           ->->->  player_mancala stones - opponment_mancala stones  
        """
        p1_score = state.board[self.p1_mancala_index]
        p2_score = state.board[self.p2_mancala_index]
        return (p1_score - p2_score) if player == 1 else (p2_score - p1_score)
    
    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        lower, upper = self.p1_pits_index
        side1_empty = all(state.board[i] == 0 for i in range(lower, upper + 1))
        lower, upper = self.p2_pits_index
        side2_empty = all(state.board[i] == 0 for i in range(lower, upper + 1))
        return side1_empty or side2_empty
    
    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move
    
    def play_game(self):
        """Play a move-alternating game."""
        print("Starting Mancala!")
        
        while True:
            '''self.display_board()'''
            state = GameState(
                to_move = self.current_player,
                utility = 0,
                board = self.board.copy(),
                moves = self.moves.copy())
            
            
            if self.current_player == 1:
                pit = alpha_beta_search(state, self, 4)
                if pit == None:
                    print("None Error")
                    return
            else: 
                pit = self.random_move_generator() # Generate a random move for either player
            
            '''print(f"Player {self.current_player} chose pit {pit}")'''
            
            self.play(pit)
            
            # Check for winning state
            win = self.winning_eval() # Will return 1 (player 1 wins), 2 (player 2 wins), 3 (tie) or 0 (no winner yet)
            if win:
                win_messages = {
                    1: "\nGAME OVER. Player 1 wins",
                    2: "\nGAME OVER. Player 2 wins",
                    3: "\nGAME OVER. Tie"
                }
                print(win_messages.get(win, "Unexpected Value"))
                print("Player 1 Mancala Value: " + str(self.board[self.p1_mancala_index]))
                print("Player 2 Mancala Value: " + str(self.board[self.p2_mancala_index]))
                return win

    def play(self, pit):
        """ 
        Execute one move: validate, move stones, capture if needed, and switch turns
        """
        # Check for valid move
        if not self.valid_move(pit): 
            print("INVALID MOVE")
            return

        # Keep track of turns for report
        if self.current_player == 1:
            self.p1_turn_amount += 1
        else: self.p2_turn_amount += 1

        # Append to moves list
        self.moves.append((self.current_player, pit))  

        # Set values for helper function
        if self.current_player == 1:
            current_mancala = self.p1_mancala_index
            opponent_mancala = self.p2_mancala_index

            start_pit = pit-1 # Account for 0-based index
        else:
            current_mancala = self.p2_mancala_index
            opponent_mancala = self.p1_mancala_index

            start_pit = pit + self.pits_per_player # Account for 0-based index and mancala index

        # Collect stones and zero the choosen pit  
        stoneCount = self.board[start_pit]
        self.board[start_pit] = 0

        # Find first pit to be dropped and distribute stones
        currentPit = start_pit + 1
        self.play_helper(stoneCount, currentPit, opponent_mancala, current_mancala)

        # Change player
        self.current_player = 2 if self.current_player == 1 else 1      
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if self.current_player == 1:
            return self.board[pit-1] > 0 and not pit > self.pits_per_player
            
        if self.current_player == 2:
            return self.board[pit+self.pits_per_player] > 0 and not pit > self.pits_per_player
             
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        while True:
            pit = random.randint(1, self.pits_per_player)
            if self.current_player == 1:   
                if self.board[pit - 1] > 0:
                    return pit
            else:
                if self.board[pit + self.pits_per_player] > 0:
                    return pit
 
    def play_helper(self, stoneCount, currentPit, opponent_mancala, current_mancala):
        """
        Helper function to distribute stones
        """
        lower, upper = (self.p1_pits_index if self.current_player == 1 else self.p2_pits_index)
        while stoneCount:
            #print("currentPit: " + str(currentPit))
            
            # Don't drop stone in opponent's mancala
            if currentPit == opponent_mancala: 
                currentPit = (currentPit + 1) % len(self.board)
                continue
                
            # Drop a stone
            self.board[currentPit] += 1
            stoneCount -= 1 
                
            # Check for capture of stones
            if stoneCount == 0 and self.board[currentPit] == 1 and currentPit in range(lower, upper+1):
                across_index = self.find_across(currentPit) # Find opposite index from current pit
                
                # Check if opposite index has stones
                if self.board[across_index] > 0:
                    self.board[current_mancala] += 1 + self.board[across_index] # Add 1 stone plus opponents stones
                    self.board[across_index] = 0 # Set opponents stones to 0
                    self.board[currentPit] = 0 # Set current pit stones to 0
                    return
                
            # Move to next pit
            currentPit = (currentPit + 1) % len(self.board)
            
                 
    def find_across(self, index):
        """
        This function finds the index directly across from the given index
        """
        if self.current_player == 1:
            return self.p2_pits_index[1] - (index - self.p1_pits_index[0])
        if self.current_player == 2:
            return self.p1_pits_index[1] - (index - self.p2_pits_index[0])
        
        return None            
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        If either of the players' pits are all empty, then it is considered a winning state.
        
        Function will return 1 (player 1 wins), 2 (player 2 wins), 3 (tie), or 0 (no winner yet)
        """
        
        player1board = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player2board = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        
        # Check for either players' pits being all empty
        if all(x == 0 for x in player1board) or all(x == 0 for x in player2board):
            # Move all of player 1's stones into mancala
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                self.board[self.p1_mancala_index] += self.board[i]
                self.board[i] = 0
            # Move all of player 2's stones into mancala
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                self.board[self.p2_mancala_index] += self.board[i]
                self.board[i] = 0
            
            # Determine the winner
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                return 1 # Player 1 wins
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                return 2 # Player 2 wins
            else: return 3 # Tie
 
        return 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
    

In [88]:
wins = {  # Initalize dictionary for keeping track of wins
    1: 0, # Player 1
    2: 0, # Player 2
    3: 0  # Tie
}

games_played = 100

p1_turn_amount, p2_turn_amount = 0, 0

for i in range(games_played):
    print("\nGAME NUMBER: " + str(i+1))
    game = Mancala(6,4)
    val = game.play_game() # Will return 1 (player 1 wins), 2 (player 2 wins), 3 (tie)
    wins[val] += 1
    p1_turn_amount += game.p1_turn_amount
    p2_turn_amount += game.p2_turn_amount
            
            
    '''
    print("\nList of valid moves:") # Printing the list of moves
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")
    '''   

    #game.display_board()


print("\nGame Statistics: Player 1")
print(f"Games Won: {wins[1] / games_played * 100:.2f}%")
print(f"Games Lost: {wins[2] / games_played * 100:.2f}%")
print(f"Games Tied: {wins[3] / games_played * 100:.2f}%")
print(f"Average Number of Turns over {games_played} games played: {p1_turn_amount/games_played}")

print("\nGame Statistics: Player 2")
print(f"Games Won: {wins[2] / games_played * 100:.2f}%")
print(f"Games Lost: {wins[1] / games_played * 100:.2f}%")
print(f"Games Tied: {wins[3] / games_played * 100:.2f}%")
print(f"Average Number of Turns over {games_played} games played: {p2_turn_amount/games_played}")


GAME NUMBER: 1
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 37
Player 2 Mancala Value: 11

GAME NUMBER: 2
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 30
Player 2 Mancala Value: 18

GAME NUMBER: 3
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 32
Player 2 Mancala Value: 16

GAME NUMBER: 4
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 31
Player 2 Mancala Value: 17

GAME NUMBER: 5
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 28
Player 2 Mancala Value: 20

GAME NUMBER: 6
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 25
Player 2 Mancala Value: 23

GAME NUMBER: 7
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 30
Player 2 Mancala Value: 18

GAME NUMBER: 8
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 37
Player 2 Mancala Value: 11

GAME NUMBER: 9
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 34
P

In [89]:
wins = {  # Initalize dictionary for keeping track of wins
    1: 0, # Player 1
    2: 0, # Player 2
    3: 0  # Tie
}

games_played = 10

p1_turn_amount, p2_turn_amount = 0, 0

for i in range(games_played):
    print("\nGAME NUMBER: " + str(i+1))
    game = Mancala(10,4)
    val = game.play_game() # Will return 1 (player 1 wins), 2 (player 2 wins), 3 (tie)
    wins[val] += 1
    p1_turn_amount += game.p1_turn_amount
    p2_turn_amount += game.p2_turn_amount
            
            
    '''
    print("\nList of valid moves:") # Printing the list of moves
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")
    '''   

    #game.display_board()


print("\nGame Statistics: Player 1")
print(f"Games Won: {wins[1] / games_played * 100:.2f}%")
print(f"Games Lost: {wins[2] / games_played * 100:.2f}%")
print(f"Games Tied: {wins[3] / games_played * 100:.2f}%")
print(f"Average Number of Turns over {games_played} games played: {p1_turn_amount/games_played}")

print("\nGame Statistics: Player 2")
print(f"Games Won: {wins[2] / games_played * 100:.2f}%")
print(f"Games Lost: {wins[1] / games_played * 100:.2f}%")
print(f"Games Tied: {wins[3] / games_played * 100:.2f}%")
print(f"Average Number of Turns over {games_played} games played: {p2_turn_amount/games_played}")


GAME NUMBER: 1
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 55
Player 2 Mancala Value: 25

GAME NUMBER: 2
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 57
Player 2 Mancala Value: 23

GAME NUMBER: 3
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 74
Player 2 Mancala Value: 6

GAME NUMBER: 4
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 60
Player 2 Mancala Value: 20

GAME NUMBER: 5
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 41
Player 2 Mancala Value: 39

GAME NUMBER: 6
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 52
Player 2 Mancala Value: 28

GAME NUMBER: 7
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 50
Player 2 Mancala Value: 30

GAME NUMBER: 8
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 61
Player 2 Mancala Value: 19

GAME NUMBER: 9
Starting Mancala!

GAME OVER. Player 1 wins
Player 1 Mancala Value: 55
Pl